In [1]:
import sys
import os
import json
from dotenv import load_dotenv
from pymongo import MongoClient
from pathlib import Path

load_dotenv(override=True)

OPENAI_API_KEY = os.environ["OPENAI_API_CHATBOT_TEST_KEY_INTERNAL"]
MONGO_URI = os.environ["MONGO_URI"]
EMBEDDING_MODEL_NAME = os.environ["EMBEDDING_MODEL_NAME"]
EMBEDDING_DIMENSIONS = os.environ["EMBEDDING_DIMENSIONS"]
CHAT_MODEL_NAME = os.environ["CHAT_MODEL_NAME"]
os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_CHATBOT_TEST_KEY_INTERNAL"]

DB_NAME = "gaia-chatbot"
COLLECTION_NAME = "documents"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
VECTOR_SIMILARITY_FUNCTION = "dotProduct"
# VECTOR_SIMILARITY_FUNCTION = "cosine"



# RETRIEVER_SEARCH_TYPE="similarity_score_threshold"
RETRIEVER_SEARCH_TYPE="mmr"

MAX_CHUNKS_TO_RETRIEVE=10
SST_CHUNK_MIN_RELEVANCE_SCORE=0.2
MMR_FETCH_K = 50
MMR_LAMBDA_MULT = 0.1



MAX_TOKENS_FOR_RESPONSE = 1000
CHAT_MODEL_TEMPERATURE=0
CHAT_MODEL_FREQ_PENALTY=0.2
CHAT_MODEL_PRES_PENALTY=0.2
SHOW_VERBOSE=False
MAX_TOKENS_FOR_HISTORY = 300


PARENT_PATH = Path.cwd().parent
EVA_SETTINGS_PATH = PARENT_PATH / 'evasettings'
EVA_SETTINGS_ENVIRONMENT_DIRECTORY = 'local'

In [2]:
models_path = PARENT_PATH / 'scripts' / 'models'
vectordatabases_path = PARENT_PATH / 'scripts' / 'vectordatabases'

if str(models_path) not in sys.path:
    sys.path.append(str(models_path))
if str(vectordatabases_path) not in sys.path:
    sys.path.append(str(vectordatabases_path))

from models import model_rag
from vectordatabases import BaseDB

In [3]:
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.runnables import RunnableSequence
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory, ChatMessageHistory
from langchain.schema import HumanMessage, SystemMessage
import re

class RAG:
    def __init__(self, chat_data):
        self.chat_data = chat_data

    ## Public Methods
    def get_response(self):
        self.llm_eva = ChatOpenAI(
            model_name=self.chat_data.rag_settings.chat_model_name,
            temperature=self.chat_data.rag_settings.temperature,
            max_tokens=self.chat_data.rag_settings.max_tokens_for_response,
            openai_api_key=self.chat_data.llm_settings.llm_key
        )

        self.summarized_history = ""
        self.memory = None
        if self.chat_data.chat_history:
            self.summarized_history, self.memory = self._summarize_history()

        # print("Summarized History: ", self.summarized_history)
        # print()
        print("User Query: ", self.chat_data.user_input)
        # print()

        base_response = self._run_base_prompt()
        if base_response is not None:
            return model_rag.ChatResponse(response=base_response, sources=[])

        # return model_rag.ChatResponse(response='', sources=[])
        
        self._detect_intent_and_rephrase_query()

        #temp
        # self.detected_intent= "other"
        #temp
        print("Detected Intent:", self.detected_intent)
        print("Rephrased User Query:", self.chat_data.user_input)
        # print("Extracted Fields:", self.extracted_fields)
        # print()
        #temp
        
        # follow_up_question = self._follow_up_questions()
        # if follow_up_question:
        #     return model_rag.ChatResponse(response=follow_up_question, sources=[])

        qa = self._get_qa_instance()
        result = qa.invoke({"question": self.chat_data.user_input})
        
        response_text, extracted_source_ids = self._format_response(result.get("answer", ""))
        source_list = self._extract_sources(result.get("source_documents", []), extracted_source_ids)
        return model_rag.ChatResponse(response=response_text, sources=source_list)

        
    ## Private Methods

    def _follow_up_questions(self):
        # Load the follow-up prompt template
        follow_up_prompt = self._load_template(
            self.chat_data.prompt_template_directory_name, 
            self.chat_data.follow_up_prompt_template_file_name
        )
        
        # Define the prompt template
        prompt_template = PromptTemplate(
            template=follow_up_prompt,
            input_variables=["missing_fields", "intent_name"]
        )
    
        # Collect missing fields
        missing_fields = []
        
        if self.detected_intent in self.chat_data.intent_details:
            intent_detail_obj = self.chat_data.intent_details[self.detected_intent]
            
            required_fields = intent_detail_obj.required_fields_prior_responding or []
            
            for field in required_fields:
                if field not in self.extracted_fields:
                    missing_fields.append(field)
    
        # If there are no missing fields, no follow-up questions are required
        if not missing_fields:
            return None 
    
        # Generate the follow-up questions using the detected intent and missing fields
        follow_up_chain = RunnableSequence(prompt_template, self.llm_eva)
        follow_up_result = follow_up_chain.invoke({
            "missing_fields": ", ".join(missing_fields),
            "intent_name": self.detected_intent
        })
    
        # Return the follow-up questions content
        return follow_up_result.content.strip()



    def _format_response(self, response_text):
        extracted_sources = []
        try:
            source_tag_pattern = r"<sources>(.*?)<\/sources>"
            matches = re.findall(source_tag_pattern, response_text, re.DOTALL)
            for match in matches:
                pans = re.split(r"\s*,\s*|\n+", match)
                pans = [pan for pan in pans if pan.isdigit()]  # Only keep numeric values
                extracted_sources.extend(pans)

            response_text = re.sub(source_tag_pattern, '', response_text).strip()
        except Exception as e:
            print(f"Error while extracting sources: {e}")

        return response_text, extracted_sources

    def _extract_sources(self, sources_documents, extracted_source_ids):
        filtered_sources_list = []
        for doc in sources_documents:
            source_id = doc.metadata.get(next((key for key in doc.metadata if key.lower() == "source"), ""), "")
            if source_id in extracted_source_ids:
                language = doc.metadata.get(next((key for key in doc.metadata if key.lower() == "language"), ""), "")
                if language.lower() == "english":
                    filtered_sources_list.append(
                        model_rag.Source(
                            source=source_id,
                            type=doc.metadata.get(next((key for key in doc.metadata if key.lower() == "type"), ""), ""),
                            title=doc.metadata.get(next((key for key in doc.metadata if key.lower() == "title"), ""), ""),
                            country=doc.metadata.get(next((key for key in doc.metadata if key.lower() == "country"), ""), ""),
                            language=language
                        )
                    )
        return filtered_sources_list
        
    
    def _load_template(self, project_template_directory_name, template_file_name):
        project_template_directory_path = os.path.join(EVA_SETTINGS_PATH, project_template_directory_name, EVA_SETTINGS_ENVIRONMENT_DIRECTORY)
        template_file_path = project_template_directory_path + '/' + template_file_name
        with open(template_file_path, "r") as file:
            return file.read()

    def _build_chat_prompt(self):
        if self.detected_intent == "other":
            chat_prompt_filename = self.chat_data.free_flowing_prompt_template_file_name
            chat_template = self._load_template(self.chat_data.prompt_template_directory_name, chat_prompt_filename)           
            return chat_template.format(
                history=self.summarized_history,
                summaries="{summaries}",
                question="{question}"
            )
        else:                        
            chat_prompt_filename = self.chat_data.intent_details.get(self.detected_intent).filename
            chat_template = self._load_template(self.chat_data.prompt_template_directory_name, chat_prompt_filename)
            key_fields = self.chat_data.intent_details[self.detected_intent].required_fields_prior_responding or ""
            
            return chat_template.format(
                history=self.summarized_history,
                key_fields=key_fields,
                summaries="{summaries}",
                question="{question}"
            )

    def _get_qa_retriever(self):
        llm_embeddings = OpenAIEmbeddings(
            model=self.chat_data.llm_settings.embedding_model_name,
            openai_api_key=self.chat_data.llm_settings.llm_key
        )
    
        db_instance = BaseDB().get_vector_db(
            self.chat_data.db_type,
            self.chat_data.db_settings,
            llm_embeddings
        )
        vector_store = db_instance.vector_index
        
        search_type = self.chat_data.rag_settings.retriever_search_settings.search_type
        search_kwargs = {
            "k": self.chat_data.rag_settings.retriever_search_settings.max_chunks_to_retrieve
        }
        if search_type == model_rag.RetrieverSearchType.Similarity_Score_Threshold:
            search_kwargs["score_threshold"] = self.chat_data.rag_settings.retriever_search_settings.retrieved_chunks_min_relevance_score
        elif search_type == model_rag.RetrieverSearchType.MMR:
            search_kwargs["fetch_k"] = self.chat_data.rag_settings.retriever_search_settings.fetch_k
            search_kwargs["lambda_mult"] = self.chat_data.rag_settings.retriever_search_settings.lambda_mult

        qa_retriever = vector_store.as_retriever(
            search_type=search_type,
            search_kwargs=search_kwargs
        )
        
        return qa_retriever


    def _get_qa_instance(self):
        chat_prompt_content = self._build_chat_prompt()
            
        prompt_template = PromptTemplate(
            template=chat_prompt_content,
            input_variables=['summaries', 'question']
        )
    
        qa_retriever = self._get_qa_retriever()

        if self.memory:
            chain_type_kwargs = {
                "verbose": SHOW_VERBOSE,
                "prompt": prompt_template,
                "memory": self.memory  # Include memory if available
            }
        else:
            chain_type_kwargs = {
                "verbose": SHOW_VERBOSE,
                "prompt": prompt_template
            }

        qa = RetrievalQAWithSourcesChain.from_chain_type(
            llm=self.llm_eva,
            chain_type="stuff",
            retriever=qa_retriever,
            return_source_documents=True,
            chain_type_kwargs=chain_type_kwargs
        )
    
        return qa


    def _get_key_fields_for_lookup(self):
        merged_key_fields = set()
        
        for intent, details in self.chat_data.intent_details.items():
            required_fields = details.required_fields_prior_responding or []
            merged_key_fields.update(required_fields)
    
        return list(merged_key_fields)
    
    def _detect_intent_and_rephrase_query(self):
        self.detected_intent = "other"
        self.extracted_fields = {}
        merged_key_fields = self._get_key_fields_for_lookup()
    
        intent_prompt = self._load_template(
            self.chat_data.prompt_template_directory_name, 
            self.chat_data.intent_detection_prompt_template_file_name
        )

        prompt_template = PromptTemplate(
            template=intent_prompt,
            input_variables=["user_input", "history", "intent_list", "merged_key_fields"]
        )
        
        intent_chain = RunnableSequence(prompt_template, self.llm_eva)
        intent_result = intent_chain.invoke({
            "user_input": self.chat_data.user_input,  
            "history": self.summarized_history,  
            "intent_list": "\n".join([f'- "{intent_name}"' for intent_name in self.chat_data.intent_details.keys()]),
            "merged_key_fields": ", ".join(merged_key_fields)
        })
        
        try:
            cleaned_content = re.sub(r"```json|```", "", intent_result.content).strip()
            intent_result_json = json.loads(cleaned_content)
            
            if all(key in intent_result_json for key in ["detected_intent", "rephrased_query", "extracted_fields"]):
                self.detected_intent = intent_result_json.get("detected_intent", self.detected_intent).strip().lower()
                self.chat_data.user_input = intent_result_json.get("rephrased_query", self.chat_data.user_input).strip()
                self.extracted_fields = intent_result_json.get("extracted_fields", self.extracted_fields)

        except (json.JSONDecodeError, AttributeError, KeyError) as e:        
            pass        
                
    
    
    def _run_base_prompt(self):
        base_prompt = self._load_template(
            self.chat_data.prompt_template_directory_name, 
            self.chat_data.base_prompt_template_file_name
        )
        
        prompt_template = PromptTemplate(
            template=base_prompt,
            input_variables=["user_input", 'history']
        )
        
        base_chain = RunnableSequence(prompt_template, self.llm_eva)
        base_result = base_chain.invoke({
            "user_input": self.chat_data.user_input,  
            "history": self.summarized_history
        })

        base_response = base_result.content.strip().strip(' "\'')
        if base_response.lower() == "none":
            return None
        return base_response

    
    def _summarize_history(self):        
        if not self.chat_data.chat_history:
            return "", None

        def trim_message(message, max_lines=2):
            lines = message.splitlines()
            if len(lines) > max_lines:
                return "\n".join(lines[:max_lines]) + "..."
            return message
        
        history = ChatMessageHistory()
        for conv in self.chat_data.chat_history:
            if conv.role.lower() == 'human':
                history.add_message(HumanMessage(content=conv.message))
            elif conv.role.lower() == 'ai':
                trimmed_message = trim_message(conv.message)
                history.add_message(SystemMessage(content=trimmed_message))

        memory_template = self._load_template(
            self.chat_data.prompt_template_directory_name, 
            self.chat_data.memory_prompt_template_file_name
        )
        
        custom_prompt = PromptTemplate(
            input_variables=['new_lines', 'summary'],
            template=memory_template
        )
        
        memory = ConversationSummaryBufferMemory(
            llm=self.llm_eva,
            max_token_limit=self.chat_data.rag_settings.max_tokens_for_history,
            prompt=custom_prompt,
            chat_memory=history,
            return_messages=True,
            memory_key="history",
            input_key="question"
        )

        memory.prune()
        summarized_history = memory.predict_new_summary(memory.chat_memory.messages, "")
       
        return summarized_history, memory



In [4]:
def generate_dummy_conversation():
    conversation_history = []

    # 1st conversation: Greeting (base template category: Introduction)
    conversation_history.append({
        "role": "Human",
        "message": "Hi!"
    })
    conversation_history.append({
        "role": "AI",
        "message": "Hello! How can I assist you today?"
    })

    # 2nd conversation: Pest list inquiry
    conversation_history.append({
        "role": "Human",
        "message": "Can you tell me what pests affect tomato crops in Kenya?"
    })
    conversation_history.append({
        "role": "AI",
        "message": "Here is a list of pests that affect tomato crops in Kenya: Tuta absoluta, whiteflies, and aphids. Would you like more information on any of these?"
    })

    # 3rd conversation: Follow-up on Tuta absoluta
    conversation_history.append({
        "role": "Human",
        "message": "Yes, can you tell me more about Tuta absoluta?"
    })
    conversation_history.append({
        "role": "AI",
        "message": "Tuta absoluta is a destructive pest for tomato crops. It lays eggs on the plant, and the larvae feed on leaves, stems, and fruits. Would you like to know how to manage it?"
    })

    # 4th conversation: Pest management (IPM)
    conversation_history.append({
        "role": "Human",
        "message": "Yes, how can I manage it?"
    })
    conversation_history.append({
        "role": "AI",
        "message": "You can use Integrated Pest Management (IPM) strategies like pheromone traps, natural predators like Trichogramma, or specific pesticides. Do you have a pesticide in mind?"
    })

    # 5th conversation: User unsure
    conversation_history.append({
        "role": "Human",
        "message": "Not really, what do you recommend?"
    })
    conversation_history.append({
        "role": "AI",
        "message": "I recommend using selective pesticides specifically targeting Tuta absoluta. Make sure to follow safety guidelines. Would you like dosage recommendations?"
    })

    # 6th conversation: User requests dosage recommendations
    conversation_history.append({
        "role": "Human",
        "message": "Yes, please!"
    })
    conversation_history.append({
        "role": "AI",
        "message": "For Tuta absoluta, apply 0.5 liters of pesticide per hectare, ensuring even coverage. Repeat every 10-14 days depending on the infestation."
    })

    # 7th conversation: Follow-up on chemical safety (Chemical Handling Safety)
    conversation_history.append({
        "role": "Human",
        "message": "What safety precautions should I take while applying the pesticide?"
    })
    conversation_history.append({
        "role": "AI",
        "message": "Wear gloves, goggles, and a mask. Store chemicals in a safe place and ensure proper disposal of containers. Do you need more advice on chemical safety?"
    })

    # 8th conversation: User responds casually (Acknowledgement messaging)
    conversation_history.append({
        "role": "Human",
        "message": "OK, sounds good!"
    })
    conversation_history.append({
        "role": "AI",
        "message": "Great! Let me know if you need further help with your crops."
    })

    # 9th conversation: Irrelevant question (Out of Scope)
    conversation_history.append({
        "role": "Human",
        "message": "By the way, do you know who won the soccer game last night?"
    })
    conversation_history.append({
        "role": "AI",
        "message": "That’s outside my expertise. I specialize in agriculture-related topics. How can I help you with your crops today?"
    })

    # 10th conversation: Diagnosis (latest context)
    conversation_history.append({
        "role": "Human",
        "message": "I need help to diagnose a problem on my tomato plants. The leaves have holes in them, and I'm not sure what's causing it."
    })
    conversation_history.append({
        "role": "AI",
        "message": "Holes in leaves can be a sign of insect pests like Tuta absoluta or leafminers. Have you noticed any larvae or eggs on the plant?"
    })

    # 11th conversation: profanity
    conversation_history.append({
        "role": "Human",
        "message": "Fuck you"
    })
    conversation_history.append({
        "role": "AI",
        "message": "I'm here to provide helpful and respectful assistance. Please refrain from using inappropriate language, biased statements, or offensive remarks."
    })

    conversation_history.append({
        "role": "Human",
        "message": "I need help to diagnose a problem."
    })

    conversation_history.append({
        "role": "AI",
        "message": "Sure! I can help. Can you please tell me which crop you need to diagnose and for what location?"
    })

    conversation_history.append({
        "role": "Human",
        "message": "Tomato"
    })

    conversation_history.append({
        "role": "AI",
        "message": "Thanks, you have chosen Tomato as a crop. May I also know your country name to give you accurate response."
    })
    
    return conversation_history




# conversation_history = []
conversation_history = generate_dummy_conversation()


def get_chatbot_response(payload: model_rag.ChatRequest):
    chat_processor = RAG(payload)
    response = chat_processor.get_response()
    return response
   

def call_chatbot_endpoint(user_input_text):
    global conversation_history

    # Directly create an instance of model_rag.ChatRequest with the required values
    chat_request = model_rag.ChatRequest(
        db_type="mongodb",
        db_settings={
            "uri": MONGO_URI,  
            "db_name": DB_NAME,  
            "collection_name": COLLECTION_NAME,  
            "vector_index_name": ATLAS_VECTOR_SEARCH_INDEX_NAME,  
            "vector_similarity_function": VECTOR_SIMILARITY_FUNCTION
        },
        llm_settings={
            "llm_key": OPENAI_API_KEY,  
            "vector_dimension_size": EMBEDDING_DIMENSIONS,  
            "embedding_model_name": EMBEDDING_MODEL_NAME
        },
        rag_settings={
            "chat_model_name": CHAT_MODEL_NAME,
            "max_tokens_for_response": MAX_TOKENS_FOR_RESPONSE,
            "retriever_search_settings": {
              "fetch_k": MMR_FETCH_K,
              "lambda_mult": MMR_LAMBDA_MULT,
              "max_chunks_to_retrieve": MAX_CHUNKS_TO_RETRIEVE,
              "retrieved_chunks_min_relevance_score": SST_CHUNK_MIN_RELEVANCE_SCORE,
              "search_type": RETRIEVER_SEARCH_TYPE
            },
            "temperature": CHAT_MODEL_TEMPERATURE,  
            "frequency_penalty": CHAT_MODEL_FREQ_PENALTY,
            "presence_penalty": CHAT_MODEL_PRES_PENALTY,
            "max_tokens_for_history": MAX_TOKENS_FOR_HISTORY
        },
        user_input=user_input_text,
        chat_history=conversation_history,
        prompt_template_directory_name="gaia",  
        base_prompt_template_file_name="base_template.txt", 
        memory_prompt_template_file_name="memory_summarizer.txt", 
        intent_detection_prompt_template_file_name="detect_intent.txt", 
        follow_up_prompt_template_file_name="follow_up.txt",
        free_flowing_prompt_template_file_name="free_flowing.txt",
        intent_details = {
            "diagnosis": {
                "filename": "diagnosis.txt",
                "description": "This intent covers queries related to diagnosing pests or problems affecting crops, including identifying potential pests or diseases based on symptoms, crop type, and location.",
                "required_fields_prior_responding": ["crop", "country/region/location", "symptoms"]
            },
            "symptoms identification": {
                "filename": "symptoms_identification.txt",
                "description": "This intent provides detailed information about symptoms caused by a specific pest or problem, including visual indicators and progression of the symptoms.",
                "required_fields_prior_responding": ["pest"]
            },
            "pest list by location": {
                "filename": "pest_list.txt",
                "description": "This intent provides a list of pests that affect a specific crop in a specific country or region.",
                "required_fields_prior_responding": ["crop", "country/region/location"]
            },
            "integrated pest management advice": {
                "filename": "ipm_pest_management.txt",
                "description": "This intent provides integrated pest management (IPM) advice, including prevention strategies, biocontrol recommendations, and chemical pesticide usage for managing pests or diseases on crops.",
                "required_fields_prior_responding": ["crop", "country/region/location", "pest"]
            },
            "chemical handling": {
                "filename": "chemical_handling_safety.txt",
                "description": "This intent provides safety advice for handling and applying specific chemicals, including personal protective equipment (PPE), safe storage, and disposal recommendations.",
                "required_fields_prior_responding": ["chemical name"]
            },
            "invasive pest status": {
                "filename": "invasive_pest_status.txt",
                "description": "This intent provides information on the current status, distribution, and spread of invasive pests in a specific country or region.",
                "required_fields_prior_responding": ["pest", "country/region/location"]
            },
            "dosage recommendations": {
                "filename": "dosage_recommendations.txt",
                "description": "This intent provides dosage recommendations for chemical or biocontrol products, including application rates, frequency, and any location-specific restrictions or precautions.",
                "required_fields_prior_responding": ["chemical name", "crop", "pest", "country/region/location", "size/area of the crop"]
            }
        }


    )

    chatbot_response = get_chatbot_response(payload=chat_request)
    
    conversation_history.append({
        "role": "Human",
        "message": user_input_text,  
    })
    conversation_history.append({
        "role": "AI",  
        "message": chatbot_response.response  
    })

    print('Bot''s Response:', chatbot_response.response)
    # print('Sources: ', chatbot_response.sources)
    print()
    print()


##### Base Cases/First Stage Testing (Dummy History is included)

In [5]:
call_chatbot_endpoint("asdasdads")
call_chatbot_endpoint("fuck you")
call_chatbot_endpoint("Wht is the captial of india")
call_chatbot_endpoint("good evening")
call_chatbot_endpoint("Thank you")
call_chatbot_endpoint("hehehe")
call_chatbot_endpoint("OK")
call_chatbot_endpoint("Why men are paid more than women?")
call_chatbot_endpoint("Shit")
call_chatbot_endpoint("You are stupid chatbot")

User Query:  asdasdads
Bots Response: It seems like your input is not clear. Could you please provide more details or ask a specific question related to agriculture?


User Query:  fuck you
Bots Response: I'm here to assist with agriculture-related queries. Please keep the conversation respectful. How can I help you with your agricultural needs?


User Query:  Wht is the captial of india
Bots Response: This query is outside the scope of agriculture advisory. I am here to assist with questions related to pest management, chemical control, crop care, and general plant advice. If you have any agriculture-related questions, feel free to ask!


User Query:  good evening
Bots Response: Hello! How can I assist you with your agriculture-related queries today?


User Query:  Thank you
Bots Response: Thank you for your message! How can I assist you with your agriculture-related queries today?


User Query:  hehehe
Bots Response: Hello! It seems like your message doesn't relate to agriculture adv

##### Diagnosis

In [16]:
call_chatbot_endpoint("""
I need help to diagnose a problem on maize in Kenya? It is affecting the leaves. holes in the leaves
""")

User Query:  
I need help to diagnose a problem on maize in Kenya? It is affecting the leaves. holes in the leaves

Detected Intent: diagnosis
Rephrased User Query: I need help to diagnose a problem on maize in Kenya. The leaves are affected and have holes in them.
Bots Response: To diagnose the issue with your maize in Kenya, I need more specific details about the symptoms. The presence of holes in the leaves can be caused by various pests or diseases. Here are a few questions to help narrow down the diagnosis:

1. **Appearance of the Holes**: Are the holes irregular or uniform in shape?
2. **Other Symptoms**: Are there any other symptoms such as discoloration, spots, or streaks on the leaves?
3. **Timing**: When did you first notice the holes? Are they spreading quickly?
4. **Environmental Conditions**: Have there been any recent changes in weather or farming practices?

Based on the initial information provided, here are a couple of potential diagnoses:

### Possible Diagnoses

1. *

##### Dosage Recommendations

In [18]:
call_chatbot_endpoint("""
Can you recommend the correct dosage of pesticide for controlling black armyworm on my maize crop in Kenya? 
I need to know how much to apply per hectar
""")

User Query:  
Can you recommend the correct dosage of pesticide for controlling black armyworm on my maize crop in Kenya? 
I need to know how much to apply per hectar

Detected Intent: dosage recommendations
Rephrased User Query: Can you recommend the correct dosage of pesticide for controlling black armyworm on my maize crop in Kenya? I need to know how much to apply per hectare.
Bots Response: To control black armyworm (Spodoptera exempta) on your maize crop in Kenya, you can use the following pesticide recommendations:

**Chemical Control**:
- **Recommended Insecticides**: Organophosphorous compounds, carbamates, and synthetic pyrethroids are effective against S. exempta. Specifically, fenitrothion and cypermethrin are mentioned as effective when applied at low dose rates by Ultra Low Volume (ULV) spraying.

**Dosage Recommendations**:
- **Fenitrothion**: Apply at a low dose rate suitable for ULV spraying. The exact dosage should be confirmed with local agricultural extension servic

##### Invasive Pest Status

In [8]:
call_chatbot_endpoint("""
What is the current status and distribution of black Armyworm in Kenya? 
I’d like to know how widespread the pest is in the region.
""")

User Query:  
What is the current status and distribution of black Armyworm in Kenya? 
I’d like to know how widespread the pest is in the region.

Detected Intent: invasive pest status
Rephrased User Query: What is the current status and distribution of black Armyworm in Kenya? I’d like to know how widespread the pest is in the region.
Bots Response: The black armyworm, *Spodoptera exempta*, is a significant pest in Kenya, particularly affecting crops such as maize, sorghum, and other cereals. The pest is known for its rapid spread and severe infestations, which can lead to total defoliation or destruction of plants to ground level.

**Current Status and Distribution in Kenya:**
- *Spodoptera exempta* is present in Kenya and is a notable pest in the region.
- The pest is known for its long-range migrations, which contribute to its rapid spread across different areas.
- Effective forecasting and management systems are in place in East Africa, including Kenya, to monitor and control the 

##### Integrated Pest Management (IPM) Advice

In [9]:
call_chatbot_endpoint("""
How can I manage the Tomato Leafminer (Tuta absoluta) in Kenya using integrated pest management strategies?
""")

User Query:  
How can I manage the Tomato Leafminer (Tuta absoluta) in Kenya using integrated pest management strategies?

Detected Intent: integrated pest management advice
Rephrased User Query: What are the integrated pest management strategies for managing Tomato Leafminer (Tuta absoluta) in Kenya?
Bots Response: To manage Tomato Leafminer (Tuta absoluta) in Kenya using integrated pest management (IPM) strategies, follow these steps:

1. **Prevention**:
   - **Crop Rotation**: Practice crop rotation with non-host plants such as maize, sorghum, or millet to disrupt the life cycle of the pest.
   - **Sanitation**: Remove plant debris to prevent it from becoming a reservoir for the pest.
   - **Mulching**: Apply mulch on the ground to prevent pupae from reaching the soil for further development.
   - **Hedge Rows**: Conserve natural enemies by providing hedge rows of flowering plants around the farm.

2. **Biocontrol Recommendations**:
   - **Natural Predators**: Conserve natural enemi

##### Pest List By Location

In [10]:
call_chatbot_endpoint("""
Can you provide a list of pests that commonly affect maize crops in Kenya?
""")

User Query:  
Can you provide a list of pests that commonly affect maize crops in Kenya?

Detected Intent: pest list by location
Rephrased User Query: Can you provide a list of pests that commonly affect maize crops in Kenya?
Bots Response: Certainly! Here is a list of pests that commonly affect maize crops in Kenya:

1. **Sitophilus zeamais (Greater Grain Weevil)**
2. **Marasmia trapezalis (Maize Webworm)**
3. **Fusarium sporotrichioides (Kernel Rot of Maize)**
4. **Pachnoda interrupta (Chafer Beetle)**
5. **Maize Rough Dwarf Virus**
6. **Physoderma maydis (Brown Spot of Corn)**
7. **Helicoverpa armigera (Cotton Bollworm)**
8. **Chilo partellus (Spotted Stem Borer)**

These pests can cause significant damage to maize crops, affecting yield and quality. For more detailed information on each pest, including symptoms and control methods, you can refer to the respective datasheets or consult local agricultural extension services.




##### Chemical Handling

In [11]:
call_chatbot_endpoint("""
What safety precautions should I take while using glyphosate on my crops?
""")

User Query:  
What safety precautions should I take while using glyphosate on my crops?

Detected Intent: chemical handling
Rephrased User Query: What safety precautions should I take while using glyphosate on my crops in Kenya?
Bots Response: When using glyphosate on your crops in Kenya, it is crucial to follow safety precautions to protect yourself, others, and the environment. Here are some general guidelines:

### Personal Protective Equipment (PPE)
- **Clothing**: Wear long-sleeved shirts and trousers that cover your legs. Ensure the trousers are pulled over the outside of your boots.
- **Gloves**: Use gloves made of nitrile, rubber, or neoprene. They should reach the elbows and fit comfortably.
- **Boots**: Wear waterproof rubber boots. Avoid leather boots as they can absorb pesticides.
- **Eye Protection**: Use glasses or a face shield to protect your eyes.
- **Face Mask**: Use a face mask to cover your mouth and nose.
- **Apron**: Use an apron, especially in wet conditions, to 

##### Symptoms Identification

In [12]:
call_chatbot_endpoint("""
What are the symptoms caused by the black armyworm on maize, and how do they progress?
""")

User Query:  
What are the symptoms caused by the black armyworm on maize, and how do they progress?

Detected Intent: symptoms identification
Rephrased User Query: What are the symptoms caused by the black armyworm on maize, and how do they progress?
Bots Response: **Symptoms Details**:
- **Initial Symptoms**: The primary symptom of Spodoptera exempta (black armyworm) attack on maize is gross feeding damage to the foliage.
- **Progression**: As the infestation progresses, the larvae feed on the growing points and young stems of the maize plants.
- **Severe Infestation**: In cases of severe infestation, the damage can lead to total defoliation or destruction of the plant down to ground level.




##### Non-English Queries (Hindi)

In [15]:
call_chatbot_endpoint("भारत में चावल पर कौन से कीट आम हैं")
call_chatbot_endpoint("चावल पर भारत से कुछ 5 to 10 जैव नियंत्रण उत्पादों का सुझाव दें")

User Query:  भारत में चावल पर कौन से कीट आम हैं
Detected Intent: pest list by location
Rephrased User Query: Can you provide a list of common pests affecting rice crops in India?
Bots Response: Certainly! Here is a list of common pests affecting rice crops in India:

1. **Brown Planthopper (Nilaparvata lugens)**
   - Major problem in lowland rice, causing "hopper burn" and transmitting rice ragged stunt virus and rice grassy stunt virus.
   

2. **Rice Field Rat (Rattus argentiventer)**
   - Causes significant damage by cutting tillers and feeding on seeds and grains.
   

3. **Rice Water Weevil (Lissorhoptrus oryzophilus)**
   - Adults damage leaves, while larvae prune roots, causing stunting and yield reduction.
   

4. **Rice Skipper (Parnara guttatus)**
   - Larvae defoliate rice plants, particularly during the reproductive growth stage.
   

5. **Rice Mealybug (Brevennia rehi)**
   - Causes stunted and sickly-looking plants with scorched appearance, known locally as 'Soorai' or 'S